In [ ]:
# 1. Configuración inicial y carga de paquetes
using Pkg
Pkg.activate(".")



In [1]:
using MLJ, DataFrames, Statistics, Random
using CSV, Dates, CategoricalArrays
using MLJLinearModels, DecisionTree, NearestNeighborModels, LightGBM



┌ Info: lib_lightgbm found in system dirs!
└ @ LightGBM C:\Users\natal\.julia\packages\LightGBM\xQZ0z\src\LightGBM.jl:32


In [ ]:
#pkg> instantiate
#pkg> update
#pkg> build SpecialFunctions


In [2]:
using Pkg
#Pkg.status(["MLJ", "DecisionTree", "MLJBase"])

In [ ]:
#Pkg.add("EvoTrees")

In [3]:
using EvoTrees

In [ ]:
# CÓDIGO COMPLETO FINAL - Con encoding de variables categóricas
using MLJ, DataFrames, CSV, Statistics, Dates, Random, CategoricalArrays
using MLJ: @load, machine, fit!, predict
using EvoTrees
using LightGBM 
import LightGBM.MLJInterface: LGBMRegressor  # define the regressor type directly

# ========================================
# 1. CARGA Y PROCESAMIENTO DE DATOS
# ========================================

# Cargar datos desde carpeta dataO
df_train = DataFrame(CSV.File("dataO/train_data.csv"))
df_week = DataFrame(CSV.File("dataO/date_to_week_id_map.csv"))
df_prices = DataFrame(CSV.File("dataO/product_prices.csv"))

# Realizar joins
df = innerjoin(df_prices, df_week, on = :week_id)
df_final = innerjoin(df_train, df, on = [:date, :product_identifier, :outlet])

# Preparar features con encoding de variables categóricas
using CategoricalArrays

# Convertir a categóricas
df_final.category_of_product = categorical(df_final.category_of_product)
df_final.state = categorical(df_final.state)

# Crear encodings
df_final.state_encoded = levelcode.(df_final.state)
df_final.category_encoded = levelcode.(df_final.category_of_product)
df_final.Month = month.(df_final.date)

# Seleccionar features incluyendo las variables categóricas encodificadas
X = select(df_final, [:product_identifier, :department_identifier, :outlet, 
                      :sell_price, :Month, :state_encoded, :category_encoded])
y = df_final.sales

# División train/test reproducible (70/30)
Random.seed!(42)
n = nrow(X)
train_size = floor(Int, 0.7 * n)
perm = randperm(n)

X_train = X[perm[1:train_size], :]
X_test = X[perm[train_size+1:end], :]
y_train = y[perm[1:train_size]]
y_test = y[perm[train_size+1:end]]


# Verificar dimensiones de los datos
data_summary = (
    total_rows = nrow(df_final),
    features = ncol(X),
    train_size = nrow(X_train),
    test_size = nrow(X_test),
    feature_names = names(X)
)

# ========================================
# 2. CARGA DE MODELOS
# ========================================

# Cargar todos los modelos funcionales
@load LinearRegressor pkg=MLJLinearModels verbosity=0
@load RidgeRegressor pkg=MLJLinearModels verbosity=0
@load LassoRegressor pkg=MLJLinearModels verbosity=0
@load KNNRegressor pkg=NearestNeighborModels verbosity=0
@load EvoTreeRegressor pkg=EvoTrees verbosity=0
#Nuevo
@load LGBMRegressor        pkg=LightGBM verbosity=0

# ========================================
# 3. DEFINICIÓN DE MODELOS
# ========================================

# Arsenal completo de modelos con diferentes configuraciones
all_models = Dict(
    # Modelos lineales
    "LinearRegression" => LinearRegressor(),
    "Ridge_0.1" => RidgeRegressor(lambda=0.1),
    "Ridge_1.0" => RidgeRegressor(lambda=1.0),
    "Ridge_10.0" => RidgeRegressor(lambda=10.0),
    "Lasso_0.1" => LassoRegressor(lambda=0.1),
    "Lasso_1.0" => LassoRegressor(lambda=1.0),
    "Lasso_10.0" => LassoRegressor(lambda=10.0),
    
    # K-Nearest Neighbors
    "KNN_5" => KNNRegressor(K=5),
    "KNN_10" => KNNRegressor(K=10),
    "KNN_15" => KNNRegressor(K=15),
    "KNN_25" => KNNRegressor(K=25),
    "KNN_35" => KNNRegressor(K=35),
    
    # Evolutionary Trees (Gradient Boosting)
    "EvoTree_50" => EvoTreeRegressor(nrounds=50),
    "EvoTree_100" => EvoTreeRegressor(nrounds=100),
    "EvoTree_200" => EvoTreeRegressor(nrounds=200),


    #Nuevo - LightGBM
    "LGBM_Default"     => LGBMRegressor(),
    "LGBM_100"         => LGBMRegressor(num_iterations=100, learning_rate=0.1, num_leaves=31),
    "LGBM_200"         => LGBMRegressor(num_iterations=200, learning_rate=0.05, num_leaves=50)
)

# ========================================
# 4. ENTRENAMIENTO Y EVALUACIÓN
# ========================================

# DataFrame para almacenar resultados
ultimate_results = DataFrame(
    Model = String[],
    MAE = Float64[],
    MSE = Float64[],
    RMSE = Float64[],
    R2 = Float64[],
    Training_Time = Float64[]
)

# Entrenar y evaluar cada modelo
for (name, model) in all_models
    start_time = time()
    
    # Entrenar modelo
    mach = machine(model, X_train, y_train)
    fit!(mach, verbosity=0)
    
    training_time = time() - start_time
    
    # Realizar predicciones
    ŷ = predict(mach, X_test)
    
    # Calcular métricas
    mae_val = mean(abs.(ŷ .- y_test))
    mse_val = mean((ŷ .- y_test).^2)
    rmse_val = sqrt(mse_val)
    r2_val = 1 - sum((y_test .- ŷ).^2) / sum((y_test .- mean(y_test)).^2)
    
    # Agregar resultados
    push!(ultimate_results, (name, mae_val, mse_val, rmse_val, r2_val, training_time))
end

# ========================================
# 5. RESULTADOS FINALES
# ========================================

# Ordenar por RMSE (menor es mejor)
final_ranking = sort(ultimate_results, :RMSE)

# Mostrar resumen de datos y ranking final
(data_summary = data_summary, ranking = final_ranking)

┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc EvoTrees.EvoTreeRegressor` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Count}}}, AbstractVector{Count}}
│ 
│ fit_data_scitype(model) = Union{Tuple{Union{Table{<:Union{AbstractVector{<:Continuous}, AbstractVector{<:Count}, AbstractVector{<:OrderedFactor}, AbstractVector{<:Multiclass}}}, AbstractMa

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002677 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 195
[LightGBM] [Info] Number of data points in the train set: 276500, number of used features: 7
[LightGBM] [Info] Start training from score 1.228510


┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc LightGBM.LGBMRegressor` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Count}}}, AbstractVector{Count}}
│ 
│ fit_data_scitype(model) = Union{Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{Continuous}}, Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{Continuous}, AbstractVect

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004144 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195
[LightGBM] [Info] Number of data points in the train set: 276500, number of used features: 7
[LightGBM] [Info] Start training from score 1.228510


┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc LightGBM.LGBMRegressor` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Count}}}, AbstractVector{Count}}
│ 
│ fit_data_scitype(model) = Union{Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{Continuous}}, Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{Continuous}, AbstractVect

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006705 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195
[LightGBM] [Info] Number of data points in the train set: 276500, number of used features: 7
[LightGBM] [Info] Start training from score 1.228510


(data_summary = (total_rows = 395000,
                 features = 7,
                 train_size = 276500,
                 test_size = 118500,
                 feature_names = ["product_identifier", "department_identifier", "outlet", "sell_price", "Month", "state_encoded", "category_encoded"],),
 ranking = 18×6 DataFrame
 Row │ Model             MAE      MSE       RMSE     R2          Training_Time ⋯
     │ String            Float64  Float64   Float64  Float64     Float64       ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ LGBM_Default      1.10681   8.11152  2.84807   0.377788       1.609     ⋯
   2 │ KNN_35            1.07139   8.16223  2.85696   0.373898       0.199
   3 │ EvoTree_200       1.10103   8.16419  2.8573    0.373748       3.879
   4 │ EvoTree_100       1.12776   8.29465  2.88004   0.36374        2.23
   5 │ KNN_25            1.06201   8.31114  2.88291   0.362475       0.183     ⋯
   6 │ LGBM_200          1.19327   8.41529  2.9

In [5]:
X

Row,product_identifier,department_identifier,outlet,sell_price,Month,state_encoded,category_encoded
,Int64,Int64,Int64,Float64,Int64,Int64,Int64
1,74,11,111,2.94,1,2,3
2,74,11,111,2.94,1,2,3
3,74,11,111,2.94,1,2,3
4,74,11,111,2.94,1,2,3
5,74,11,111,2.94,1,2,3
6,74,11,111,2.94,1,2,3
7,74,11,111,2.94,1,2,3
8,74,11,111,2.94,1,2,3
9,74,11,111,2.94,1,2,3


In [48]:

using MLJ, DataFrames, CSV, Statistics, Dates, Random, CategoricalArrays
using MLJ: @load, machine, fit!, predict
using EvoTrees
import DataFrames: transform

# ========================================
# 1. CARGA Y PREPARACIÓN INICIAL
# ========================================

df_train = DataFrame(CSV.File("dataO/train_data.csv"))
df_week = DataFrame(CSV.File("dataO/date_to_week_id_map.csv"))
df_prices = DataFrame(CSV.File("dataO/product_prices.csv"))

# Joins iniciales
df = innerjoin(df_prices, df_week, on = :week_id)
df_final = innerjoin(df_train, df, on = [:date, :product_identifier, :outlet])

# ========================================
# 2. FEATURES TEMPORALES AVANZADAS
# ========================================

# Features de tiempo básicas
df_final.Month = month.(df_final.date)
df_final.DayOfWeek = dayofweek.(df_final.date)
df_final.DayOfMonth = day.(df_final.date)
df_final.Quarter = quarterofyear.(df_final.date)
df_final.WeekOfYear = week.(df_final.date)

# Features estacionales
df_final.IsWeekend = df_final.DayOfWeek .>= 6
df_final.IsMonthStart = df_final.DayOfMonth .<= 7
df_final.IsMonthEnd = df_final.DayOfMonth .>= 25
df_final.IsHolidayMonth = (df_final.Month .== 12) .|| (df_final.Month .== 1)  # Diciembre/Enero

# Features cíclicas (importantes para capturar patrones estacionales)
df_final.Month_sin = sin.(2π * df_final.Month / 12)
df_final.Month_cos = cos.(2π * df_final.Month / 12)
df_final.DayOfWeek_sin = sin.(2π * df_final.DayOfWeek / 7)
df_final.DayOfWeek_cos = cos.(2π * df_final.DayOfWeek / 7)

# ========================================
# 3. FEATURES DE PRECIOS
# ========================================

# Features de precios por grupos
price_stats = combine(groupby(df_final, :product_identifier), 
    :sell_price => mean => :price_product_mean,
    :sell_price => std => :price_product_std,
    :sell_price => minimum => :price_product_min,
    :sell_price => maximum => :price_product_max
)

outlet_price_stats = combine(groupby(df_final, :outlet),
    :sell_price => mean => :price_outlet_mean,
    :sell_price => std => :price_outlet_std
)

category_price_stats = combine(groupby(df_final, :category_of_product),
    :sell_price => mean => :price_category_mean,
    :sell_price => std => :price_category_std
)

# Unir estadísticas de precios
df_final = leftjoin(df_final, price_stats, on = :product_identifier)
df_final = leftjoin(df_final, outlet_price_stats, on = :outlet)
df_final = leftjoin(df_final, category_price_stats, on = :category_of_product)

# Features derivadas de precios
df_final.price_vs_product_mean = df_final.sell_price ./ df_final.price_product_mean
df_final.price_vs_outlet_mean = df_final.sell_price ./ df_final.price_outlet_mean
df_final.price_vs_category_mean = df_final.sell_price ./ df_final.price_category_mean

# Reemplazar NaN y valores infinitos con 1.0
#for col in [:price_vs_product_mean, :price_vs_outlet_mean, :price_vs_category_mean]
#    df_final[!, col] = replace(df_final[!, col], NaN => 1.0, Inf => 1.0, -Inf => 1.0)
#end

# Limpieza rápida de NaN/Inf:
for col in [:price_vs_product_mean, :price_vs_outlet_mean, :price_vs_category_mean]
    df_final[!, col] = [ (v isa Missing || isnan(v) || isinf(v)) ? 1.0 : v
                         for v in df_final[!, col] ]
end



# ========================================
# 4. FEATURES DE VENTAS HISTÓRICAS (LAG FEATURES)
# ========================================

# Ordenar por producto, outlet y fecha
sort!(df_final, [:product_identifier, :outlet, :date])

# Lag features de ventas (ventas de días anteriores)
df_final.sales_lag_1 = [missing; df_final.sales[1:end-1]]
df_final.sales_lag_7 = [fill(missing, 7); df_final.sales[1:end-7]]

# Promedios móviles de ventas
function rolling_mean(x, window)
    result = similar(x, Float64)
    for i in 1:length(x)
        start_idx = max(1, i - window + 1)
        result[i] = mean(x[start_idx:i])
    end
    return result
end

df_final.sales_rolling_3 = rolling_mean(df_final.sales, 3)
df_final.sales_rolling_7 = rolling_mean(df_final.sales, 7)
df_final.sales_rolling_30 = rolling_mean(df_final.sales, 30)

# NUEVAS FEATURES

# ========================================
# 5. FEATURES ADICIONALES
# ========================================

# 5.1 Price & sales lag de 7 días
df_final.sell_price_lag_7 = [fill(missing, 7); df_final.sell_price[1:end-7]]
df_final.sales_lag_7      = [fill(missing, 7); df_final.sales[1:end-7]]

# 5.2 Cambios porcentuales a 7 días
df_final.price_pct_change_7 = [
    (ismissing(p) || p == 0) ? 0.0 : (df_final.sell_price[i] - p) / p 
    for (i, p) in enumerate(df_final.sell_price_lag_7)
]
df_final.sales_pct_change_7 = [
    (ismissing(s) || s == 0) ? 0.0 : (df_final.sales[i] - s) / s 
    for (i, s) in enumerate(df_final.sales_lag_7)
]

# 5.3 Elasticidad precio — respuesta de ventas al cambio de precio
df_final.price_elasticity = [
    pct == 0 ? 0.0 : sale_pct / pct
    for (pct, sale_pct) in zip(df_final.price_pct_change_7, df_final.sales_pct_change_7)
]

# 5.4 Agregados semanales por producto-outlet
# 1) Creas el DataFrame “weekly”

#weekly = combine(groupby(df_final, [:product_identifier, :outlet, :week_id]),
#                 :sales => sum => :weekly_sales)
#sort!(weekly, [:product_identifier, :outlet, :week_id])

# 2) paso 1: creas lag1
#weekly = transform(groupby(weekly, [:product_identifier, :outlet]),
#    :weekly_sales => (x->[missing; x[1:end-1]]) => :weekly_sales_lag1
#)

# 3) paso 2: creas growth (ahora :weekly_sales_lag1 ya existe)
#weekly = transform(groupby(weekly, [:product_identifier, :outlet]),
#    [:weekly_sales, :weekly_sales_lag1] => ((s,l)->
#      [ (ismissing(li)||li==0) ? 0.0 : (si - li)/li
#        for (si, li) in zip(s,l) ]
#    ) => :weekly_growth
#)

# Unir back a df_final
#df_final = leftjoin(df_final, weekly,
#    on=[:product_identifier, :outlet, :week_id])

# 5.5 Agregados mensuales por producto-outlet
monthly = combine(groupby(df_final, [:product_identifier, :outlet, :Month]),
                  :sales => sum => :monthly_sales)
sort!(monthly, [:product_identifier, :outlet, :Month])

# lag mensual y crecimiento
#monthly.monthly_sales_lag1 = similar(monthly.monthly_sales)
#for g in groupby(monthly, [:product_identifier, :outlet])
#    idxs = g.rows
#    monthly.monthly_sales_lag1[idxs] = [missing; g.monthly_sales[1:end-1]]
#end
#monthly.monthly_growth = [
#    (ismissing(l) || l == 0) ? 0.0 : (m - l) / l
#    for (m, l) in zip(monthly.monthly_sales, monthly.monthly_sales_lag1)
#]

# Unir back a df_final
df_final = leftjoin(df_final, monthly,
    on=[:product_identifier, :outlet, :Month])







# ========================================
# 5. FEATURES AGREGADAS POR ENTIDADES
# ========================================

# Estadísticas por producto
product_stats = combine(groupby(df_final, :product_identifier),
    :sales => mean => :product_sales_mean,
    :sales => std => :product_sales_std,
    :sales => sum => :product_total_sales,
    nrow => :product_frequency
)

# Estadísticas por outlet
outlet_stats = combine(groupby(df_final, :outlet),
    :sales => mean => :outlet_sales_mean,
    :sales => std => :outlet_sales_std,
    :sales => sum => :outlet_total_sales,
    nrow => :outlet_frequency
)

# Estadísticas por categoría
category_stats = combine(groupby(df_final, :category_of_product),
    :sales => mean => :category_sales_mean,
    :sales => std => :category_sales_std,
    :sales => sum => :category_total_sales
)

# Estadísticas por estado
state_stats = combine(groupby(df_final, :state),
    :sales => mean => :state_sales_mean,
    :sales => std => :state_sales_std,
    :sales => sum => :state_total_sales
)

# Unir todas las estadísticas
df_final = leftjoin(df_final, product_stats, on = :product_identifier)
df_final = leftjoin(df_final, outlet_stats, on = :outlet)
df_final = leftjoin(df_final, category_stats, on = :category_of_product)
df_final = leftjoin(df_final, state_stats, on = :state)

# ========================================
# 6. FEATURES DE INTERACCIONES
# ========================================

# Interacciones importantes
df_final.outlet_x_category = df_final.outlet .* 1000 .+ hash.(df_final.category_of_product) .% 1000
df_final.state_x_category = hash.(df_final.state) .% 100 .* 1000 .+ hash.(df_final.category_of_product) .% 1000
df_final.month_x_category = df_final.Month .* 1000 .+ hash.(df_final.category_of_product) .% 1000

# Features de densidad/competencia
df_final.products_per_outlet = df_final.outlet_frequency ./ length(unique(df_final.product_identifier))
df_final.outlets_per_state = [length(unique(df_final[df_final.state .== s, :outlet])) for s in df_final.state]

# ========================================
# 7. ENCODING DE VARIABLES CATEGÓRICAS
# ========================================

# Convertir a categóricas
df_final.category_of_product = categorical(df_final.category_of_product)
df_final.state = categorical(df_final.state)

# Crear encodings
df_final.state_encoded = levelcode.(df_final.state)
df_final.category_encoded = levelcode.(df_final.category_of_product)



# ========================================
# 8. SELECCIÓN DE FEATURES FINALES
# ========================================

# Eliminar filas con missing values en lag features (para simplificar)
df_final = df_final[.!ismissing.(df_final.sales_lag_1), :]

# Seleccionar features finales
feature_columns = [
    # Features básicas
    :product_identifier, :department_identifier, :outlet, :sell_price,
    
    # Features temporales
    :Month, :DayOfWeek, :DayOfMonth, :Quarter, :WeekOfYear,
    :IsWeekend, :IsMonthStart, :IsMonthEnd, :IsHolidayMonth,
    :Month_sin, :Month_cos, :DayOfWeek_sin, :DayOfWeek_cos,
    
    # Features de precios
    :price_product_mean, :price_outlet_mean, :price_category_mean,
    :price_vs_product_mean, :price_vs_outlet_mean, :price_vs_category_mean,
    
    # Features de ventas históricas
    :sales_lag_1, :sales_rolling_3, :sales_rolling_7, :sales_rolling_30,
    
    # Features agregadas
    :product_sales_mean, :product_frequency, :outlet_sales_mean, :outlet_frequency,
    :category_sales_mean, :state_sales_mean,
    
    # Features de interacciones
    :outlet_x_category, :state_x_category, :month_x_category,
    :products_per_outlet, :outlets_per_state,
    
    # Encodings categóricos
    :state_encoded, :category_encoded

    #NUEVAS
    #:sales_lag_1, 
    #:sales_lag_7,
    #:sell_price_lag_7,
    #:price_pct_change_7, :sales_pct_change_7,
    #:price_elasticity,
    #:weekly_sales, :weekly_sales_lag1, :weekly_growth,
    #:monthly_sales, #:monthly_sales_lag1, 
    #:monthly_growth,

]

X_engineered = df_final[!, feature_columns]
y_engineered = df_final.sales

# Verificar que no hay missing values
for col in names(X_engineered)
    if any(ismissing.(X_engineered[!, col]))
        println("Warning: Missing values en $col")
    end
end

# ========================================
# 9. DIVISIÓN Y EVALUACIÓN
# ========================================

# División train/test
Random.seed!(42)
n = nrow(X_engineered)
train_size = floor(Int, 0.7 * n)
perm = randperm(n)

X_train = X_engineered[perm[1:train_size], :]
X_test = X_engineered[perm[train_size+1:end], :]
y_train = y_engineered[perm[1:train_size]]
y_test = y_engineered[perm[train_size+1:end]]

# Información sobre las features
feature_info = (
    total_features = ncol(X_engineered),
    total_samples = nrow(X_engineered),
    train_samples = nrow(X_train),
    test_samples = nrow(X_test),
    feature_names = names(X_engineered)
)

feature_info

(total_features = 40,
 total_samples = 394999,
 train_samples = 276499,
 test_samples = 118500,
 feature_names = ["product_identifier", "department_identifier", "outlet", "sell_price", "Month", "DayOfWeek", "DayOfMonth", "Quarter", "WeekOfYear", "IsWeekend"  …  "outlet_frequency", "category_sales_mean", "state_sales_mean", "outlet_x_category", "state_x_category", "month_x_category", "products_per_outlet", "outlets_per_state", "state_encoded", "category_encoded"],)

In [49]:
missing_summary = Dict()
for col in names(X_train)
    missing_count = sum(ismissing.(X_train[!, col]))
    if missing_count > 0
        missing_summary[col] = missing_count
    end
end

missing_summary

Dict{Any, Any}()

In [ ]:
# Convertir todos los datos a tipos numéricos estándar (homogeniza los datos)
function fix_data_types(X, y)
    X_fixed = DataFrame()
    
    for col in names(X)
        if eltype(X[!, col]) <: Union{Missing, Number}
            # Convertir a Float64, reemplazando missing con 0.0
            X_fixed[!, col] = convert(Vector{Float64}, coalesce.(X[!, col], 0.0))
        else
            # Para otros tipos, convertir directamente
            X_fixed[!, col] = convert(Vector{Float64}, X[!, col])
        end
    end
    
    y_fixed = convert(Vector{Float64}, y)
    
    return X_fixed, y_fixed
end

# Aplicar la corrección
X_train_fixed, y_train_fixed = fix_data_types(X_train, y_train)
X_test_fixed, y_test_fixed = fix_data_types(X_test, y_test)

# Verificar tipos
(
    X_train_types = eltype.(eachcol(X_train_fixed)),
    y_train_type = eltype(y_train_fixed),
    dimensions = (nrow(X_train_fixed), ncol(X_train_fixed))
)

(X_train_types = DataType[Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64  …  Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64],
 y_train_type = Float64,
 dimensions = (276499, 40),)

In [ ]:
#Pkg.add("MLJLightGBMInterface")

In [51]:
# Modelos completos con feature engineering

using MLJ
#using  MLJLightGBMInterface


fe_models = Dict(
    "LinearRegression_FE" => LinearRegressor(),
    "Ridge_1.0_FE" => RidgeRegressor(lambda=1.0),
    "KNN_25_FE" => KNNRegressor(K=25),
    "KNN_35_FE" => KNNRegressor(K=35),
    "EvoTree_100_FE" => EvoTreeRegressor(nrounds=100),
    "EvoTree_200_FE" => EvoTreeRegressor(nrounds=200),
    # LightGBM con feature engineering
    "LGBM_Default_FE"     => LGBMRegressor(),
    "LGBM_100_FE"         => LGBMRegressor(
                               num_iterations=100,
                               learning_rate=0.1,
                               num_leaves=31
                            ),
    "LGBM_200_FE"         => LGBMRegressor(
                               num_iterations=200,
                               learning_rate=0.05,
                               num_leaves=50
                            )
)


# Evaluar todos
results_final_fe = DataFrame(
    Model = String[],
    MAE = Float64[],
    RMSE = Float64[],
    R2 = Float64[],
    Features = Int64[]
)

for (name, model) in fe_models
    mach = machine(model, X_train_fixed, y_train_fixed)
    fit!(mach, verbosity=0)
    ŷ = predict(mach, X_test_fixed)
    
    mae_val = mean(abs.(ŷ .- y_test_fixed))
    rmse_val = sqrt(mean((ŷ .- y_test_fixed).^2))
    r2_val = 1 - sum((y_test_fixed .- ŷ).^2) / sum((y_test_fixed .- mean(y_test_fixed)).^2)
    
    push!(results_final_fe, (name, mae_val, rmse_val, r2_val, ncol(X_train_fixed)))
end

sort(results_final_fe, :RMSE)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017226 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1837
[LightGBM] [Info] Number of data points in the train set: 276499, number of used features: 37
[LightGBM] [Info] Start training from score 1.227494
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013370 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1837
[LightGBM] [Info] Number of data points in the train set: 276499, number of used features: 37
[LightGBM] [Info] Start training from score 1.227494
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016879 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory i

Row,Model,MAE,RMSE,R2,Features
,String,Float64,Float64,Float64,Int64
1,LGBM_Default_FE,0.594825,1.62456,0.775698,40
2,LGBM_100_FE,0.594825,1.62456,0.775698,40
3,LGBM_200_FE,0.591386,1.63568,0.772614,40
4,LinearRegression_FE,0.655862,1.65888,0.766118,40
5,Ridge_1.0_FE,0.713618,1.81293,0.720664,40
6,EvoTree_100_FE,0.619101,1.90706,0.690904,40
7,EvoTree_200_FE,0.624935,1.95064,0.676617,40
8,KNN_25_FE,0.993251,2.35886,0.527102,40
9,KNN_35_FE,1.02405,2.45056,0.489617,40


In [ ]:
#Pkg.add("MLJTuning")

   Resolving package versions...
    Updating `C:\Users\natal\.julia\environments\v1.10\Project.toml`
  [03970b2e] + MLJTuning v0.8.8
  No Changes to `C:\Users\natal\.julia\environments\v1.10\Manifest.toml`


In [18]:
Pkg.add("BayesianOptimization")

   Resolving package versions...
    Updating `C:\Users\natal\.julia\environments\v1.10\Project.toml`
  [4c6ed407] + BayesianOptimization v0.2.5
    Updating `C:\Users\natal\.julia\environments\v1.10\Manifest.toml`
  [4c6ed407] + BayesianOptimization v0.2.5
  [6e4b80f9] + BenchmarkTools v1.6.0
  [523fee87] + CodecBzip2 v0.8.5
  [fdbdab4c] + ElasticArrays v1.2.12
  [2904ab23] + ElasticPDMats v0.2.3
  [e2ba6199] + ExprTools v0.1.10
⌅ [442a2c76] + FastGaussQuadrature v0.4.9
⌅ [f6369f11] ↓ ForwardDiff v1.0.1 ⇒ v0.10.38
  [891a1506] + GaussianProcesses v0.12.5
  [076d061b] - HashArrayMappedTries v0.2.0
  [34004b35] - HypergeometricFunctions v0.3.28
⌅ [92d709cd] ↓ IrrationalConstants v0.2.4 ⇒ v0.1.1
  [0f8b85d8] + JSON3 v1.14.3
  [b8f27783] + MathOptInterface v1.42.0
  [fdba3010] + MathProgBase v0.7.8
  [d8a4904e] + MutableArithmetics v1.6.4
⌅ [76087f3c] + NLopt v0.6.5
⌃ [872c559c] ↓ NNlib v0.9.30 ⇒ v0.9.28
⌅ [79098fc4] ↓ Rmath v0.8.0 ⇒ v0.7.1
  [7e506255] - ScopedValues v1.3.0
  [ed01d8cd] 

In [ ]:
#Pkg.add("TreeParzen")

   Resolving package versions...
    Updating `C:\Users\natal\.julia\environments\v1.10\Project.toml`
⌃ [31c24e10] ↓ Distributions v0.25.120 ⇒ v0.25.45
  [eb66a70c] + TreeParzen v0.3.4
    Updating `C:\Users\natal\.julia\environments\v1.10\Manifest.toml`
  [66dad0bd] - AliasTables v1.1.3
  [9e28174c] + BinDeps v1.0.2
  [b99e7846] + BinaryProvider v0.5.10
  [a2441757] + Coverage v1.6.1
  [c36e975a] + CoverageTools v1.3.2
  [b429d917] + DensityInterface v0.4.0
⌃ [b552c78f] ↓ DiffRules v1.15.1 ⇒ v1.7.0
⌃ [31c24e10] ↓ Distributions v0.25.120 ⇒ v0.25.45
⌅ [1a297f60] ↓ FillArrays v1.13.0 ⇒ v0.12.8
  [92d709cd] ↑ IrrationalConstants v0.1.1 ⇒ v0.2.4
⌃ [d3d80556] ↓ LineSearches v7.4.0 ⇒ v7.1.1
⌅ [77ba4419] ↓ NaNMath v1.1.3 ⇒ v0.3.7
⌃ [429524aa] ↓ Optim v1.13.2 ⇒ v1.12.0
  [43287f4e] - PtrArrays v1.3.0
⌅ [276daf66] ↓ SpecialFunctions v1.8.8 ⇒ v0.8.0
⌅ [4c63d2b9] ↓ StatsFuns v0.9.18 ⇒ v0.9.7
  [eb66a70c] + TreeParzen v0.3.4
  [30578b45] + URIParser v0.4.1
  [efe28fd5] - OpenSpecFun_jll v0.5.6+0
 

In [31]:
using MLJ
using MLJTuning
using TreeParzen: HP, MLJTreeParzenTuning
using MLJ: @load, machine, fit!, predict, CV, rms
using LightGBM; import LightGBM.MLJInterface: LGBMRegressor
# ——————————————
# 6. TUNING “BAYESIANO” PARA LIGHTGBM
# ——————————————

# 6.1. Asegúrate de tener instalado TreeParzen.jl:
# ] add TreeParzen

@load LGBMRegressor pkg=LightGBM verbosity=0

# 6.2. Definir modelo base y rangos de búsqueda
lgbm = LGBMRegressor()

# 2. Define el espacio de búsqueda usando HP.*
space = Dict(
  :num_iterations => HP.QuantUniform(:num_iterations, 50., 300., 1.),
  :learning_rate  => HP.Uniform(:learning_rate, 0.01, 0.2),
  :num_leaves     => HP.QuantUniform(:num_leaves,      10., 100., 1.)
)

#ranges = [
#    range(lgbm, :num_iterations, lower=50,  upper=300),
#    range(lgbm, :learning_rate,   lower=0.01, upper=0.2),
#    range(lgbm, :num_leaves,      lower=10,   upper=100),
#]

# 6.3. Configurar TunedModel con Tree-Parzen (bayesiano)
tuner = TunedModel(
  model       = lgbm,
  range       = space,                       # <- ojo: range=, no ranges=
  tuning      = MLJTreeParzenTuning(),       # estrategia TPE
  resampling  = CV(nfolds=5, shuffle=true),
  measure     = rms,
  n           = 25,                          # iteraciones totales
  acceleration= CPUThreads()
)

# 1) Entrena el tuner
tmach = machine(tuner, X_train_fixed, y_train_fixed)
fit!(tmach, verbosity=1)            # → que llegue a completarse sin errores

# 2) Extrae los mejores hiperparámetros
best_lgbm = fitted_params(tmach).best_model
println("Mejores hiperparámetros bayesianos para LGBM:")
println(best_lgbm)

# 3) Predice sobre el test set
ŷ = predict(tmach, X_test_fixed) |> collect

# 4) Calcula RMSE y MAE
rmse = sqrt(mean((ŷ .- y_test_fixed).^2))
mae  = mean(abs.(ŷ .- y_test_fixed))

println("\nMétricas en test:")
println(" RMSE = ", round(rmse, digits=4))
println(" MAE  = ", round(mae,  digits=4))

┌ Info: Training machine(DeterministicTunedModel(model = LGBMRegressor(objective = regression, …), …), …).
└ @ MLJBase C:\Users\natal\.julia\packages\MLJBase\7nGJF\src\machines.jl:499
┌ Info: Attempting to evaluate 25 models.
└ @ MLJTuning C:\Users\natal\.julia\packages\MLJTuning\vMe8s\src\tuned_models.jl:762
Evaluating over 20 metamodels:  10%[==>                      ]  ETA: 0:04:47

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.021850 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1848
[LightGBM] [Info] Number of data points in the train set: 221199, number of used features: 37
[LightGBM] [Info] Start training from score 1.227248
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019506 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1834
[LightGBM] [Info] Number of data points in the train set: 221199, number of used features: 37
[LightGBM] [Info] Start training from score 1.227456
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020015 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1835
[LightGBM] [Info] Number of data points in the train set: 221199, number of used features: 37
[LightGBM] [Inf

Evaluating over 20 metamodels:  15%[===>                     ]  ETA: 0:04:33

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019821 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1848
[LightGBM] [Info] Number of data points in the train set: 221199, number of used features: 37
[LightGBM] [Info] Start training from score 1.227248
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008113 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1834
[LightGBM] [Info] Number of data points in the train set: 221199, number of used features: 37
[LightGBM] [Info] Start training from score 1.227456
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013519 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1835
[LightGBM] [Info] Number of data points in th

Evaluating over 20 metamodels:  20%[=====>                   ]  ETA: 0:05:08

...

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018666 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1848
[LightGBM] [Info] Number of data points in the train set: 221199, number of used features: 37
[LightGBM] [Info] Start training from score 1.227248
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013477 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1834
[LightGBM] [Info] Number of data points in the train set: 221199, number of used features: 37
[LightGBM] [Info] Start training from score 1.227456
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014142 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1835
[LightGBM] [Info] Number of data points in the train set: 221199, number of used features: 37
[LightGBM] [Inf

Evaluating over 20 metamodels:  95%[=======================> ]  ETA: 0:00:29

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009184 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1848
[LightGBM] [Info] Number of data points in the train set: 221199, number of used features: 37
[LightGBM] [Info] Start training from score 1.227248
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014730 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1834
[LightGBM] [Info] Number of data points in the train set: 221199, number of used features: 37
[LightGBM] [Info] Start training from score 1.227456
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013787 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1835
[LightGBM] [Info] Number of data points in th

Evaluating over 20 metamodels: 100%[=========================] Time: 0:10:09


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020784 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1848
[LightGBM] [Info] Number of data points in the train set: 221199, number of used features: 37
[LightGBM] [Info] Start training from score 1.227248
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013589 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1834
[LightGBM] [Info] Number of data points in the train set: 221199, number of used features: 37
[LightGBM] [Info] Start training from score 1.227456
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008173 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1835
[LightGBM] [Info] Number of data points in th

In [32]:
using MLJ, DataFrames

# Entrenar Ridge
model_ridge = RidgeRegressor(lambda=1.0)
mach_ridge  = machine(model_ridge, X_train_fixed, y_train_fixed)
fit!(mach_ridge, verbosity=0)

# Obtener pares (feature => coef)
coef_pairs = fitted_params(mach_ridge).coefs

# Extraer nombres y valores absolutos
features = String[]
coefs    = Float64[]
for (feat,coef) in coef_pairs
    push!(features, string(feat))
    push!(coefs, abs(coef))
end

# Calcular ranking de importancia
ranks = sortperm(coefs, rev=true)

# Crear DataFrame
feature_importance = DataFrame(
    Feature         = features,
    Coefficient     = coefs,
    Importance_Rank = ranks
)

# Mostrar top 10 por coeficiente
first(sort(feature_importance, :Coefficient, rev=true), 10)


Row,Feature,Coefficient,Importance_Rank
,String,Float64,Int64
1,sales_rolling_3,0.860157,40
2,sales_rolling_7,0.199064,22
3,sales_lag_1,0.182801,13
4,sales_rolling_30,0.077704,7
5,DayOfWeek,0.0490341,28
6,product_sales_mean,0.0356142,19
7,DayOfWeek_sin,0.0211705,20
8,DayOfWeek_cos,0.0157785,12
9,IsWeekend,0.0157271,9


In [ ]:
#Pkg.add("MLJLightGBMInterface")

In [ ]:
using MLJ
using LightGBM, DataFrames

# Importar explícitamente
const LGBMRegressor = LightGBM.MLJInterface.LGBMRegressor

model = LGBMRegressor(
    objective = "regression",
    learning_rate = 0.1,
    num_iterations = 100,
    num_leaves = 31,
    metric = ["l2"] # 👈 CORREGIDO: vector de string
)

#X = MLJ.table(X_train_fixed)
y = y_train_vector

#mach = machine(model, X, y)

mach = machine(model, X_train_fixed, y)

fit!(mach)

X_test_df = DataFrame(X_test_matrix, names(X_train_fixed))

ŷ_native = predict(mach, MLJ.table(X_test_df)) |> collect

mae_native = mean(abs.(ŷ_native .- y_test_vector))
r2_native = 1 - sum((y_test_vector .- ŷ_native).^2) / sum((y_test_vector .- mean(y_test_vector)).^2)

("LightGBM_MLJ", "SUCCESS", mae_native, r2_native)


In [ ]:
using Pkg
# Pkg.status()